# Bachelor Thesis: Predicting flight delays
#### Author: Tygo Francissen, s1049742, Radboud University
This thesis aims to predict flight delays in the United States and Brazil by using a broad scale of machine learning algorithms and transfer learning.

### 1.1 Gathering the data
The data sets are gathered from the [Bureau of Transportation Statistics](https://www.transtats.bts.gov) and the [VRA](https://sas.anac.gov.br/sas/bav/view/frmConsultaVRA). First, we extract the data and have a look at it:

In [54]:
from deep_translator import GoogleTranslator
import pandas as pd

# Store the data sets for January 2022
dataBrazil = pd.read_excel('../Data/VRA_20230228124648.xlsx')
dataUSA = pd.read_csv('../Data/T_ONTIME_REPORTING.csv')

# Translate the column names of the Brazilian data set
for i, _ in enumerate(dataBrazil.columns.values):
    dataBrazil.columns.values[i] = GoogleTranslator(source='auto', target='en').translate(dataBrazil.columns.values[i])